<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Import-&amp;-Load-data" data-toc-modified-id="Import-&amp;-Load-data-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Import &amp; Load data</a></span></li></ul></li><li><span><a href="#Feature-Engineering" data-toc-modified-id="Feature-Engineering-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Feature Engineering</a></span><ul class="toc-item"><li><span><a href="#Split-training-and-test-set" data-toc-modified-id="Split-training-and-test-set-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Split training and test set</a></span></li><li><span><a href="#Missing-values" data-toc-modified-id="Missing-values-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Missing values</a></span><ul class="toc-item"><li><span><a href="#Categorical-variables" data-toc-modified-id="Categorical-variables-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Categorical variables</a></span></li><li><span><a href="#Numerical-variables" data-toc-modified-id="Numerical-variables-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Numerical variables</a></span></li></ul></li><li><span><a href="#Numeric-variable-transformation" data-toc-modified-id="Numeric-variable-transformation-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Numeric variable transformation</a></span></li><li><span><a href="#Encoding-of-categorical-variables" data-toc-modified-id="Encoding-of-categorical-variables-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Encoding of categorical variables</a></span></li></ul></li><li><span><a href="#Feature-Scaling" data-toc-modified-id="Feature-Scaling-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Feature Scaling</a></span></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Conclusion</a></span></li></ul></div>

# TMDB_Feature Engineering and Feature Scaling

### Introduction

We've cleaned data and explored data, and created new variables in the previous notebooks (TMDB_Cleaning_FeatureEngineering.ipynb and TMDB_EDA_FeatureEngineering).

In this notebook, I will perform feature engineering and feature scaling. 

### Import & Load data

In [34]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', 100)

import warnings
warnings.filterwarnings('ignore')

# to save the trained scaler class
import joblib 

from pathlib import Path
import os
os.getcwd()

'/Users/yejiseoung/Dropbox/My Mac (Yejis-MacBook-Pro.local)/Documents/Projects/TMDB'

In [2]:
# for the model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

from sklearn.pipeline import Pipeline

# for feature engineering
from feature_engine import encoding as ce

# feature scaling
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [3]:
path = Path('/Users/yejiseoung/Dropbox/My Mac (Yejis-MacBook-Pro.local)/Documents/Projects/TMDB/datasets/')

In [4]:
data = pd.read_csv(path/'engineered_train.csv')
print(data.shape)
data.head()

(3000, 23)


,id,budget,popularity,runtime,revenue,is_collection,num_genres,is_homepage,is_English,num_pro_comps,num_pro_count,num_spoken_lang,num_keywords,num_cast,num_crew,release_month,release_day,release_year,release_dayofweek,budget_year_ratio,runtime_to_mean_year,popularity_to_mean_year,budget_to_mean_year
0,1,14000000.0,6.575393,93.0,12314651,1,1,0,1,3,1,1,<5,24,medium,2,20,2015,4,3.448085,0.830821,0.642915,0.324913
1,2,40000000.0,8.248895,113.0,95149435,1,4,0,1,1,1,1,<5,20,small,8,6,2004,4,9.960120,1.075080,0.984041,1.014210
2,3,3300000.0,64.299990,105.0,13092000,0,1,1,1,3,1,1,<15,51,medium,10,10,2014,4,0.813570,0.979870,6.073148,0.091796
3,4,1200000.0,3.174936,122.0,16000000,0,2,1,0,NaN,1,2,<10,7,small,3,9,2012,4,0.296432,1.133127,0.387724,0.030480
4,5,NaN,1.148070,118.0,3923970,0,2,0,0,NaN,1,1,<5,4,small,2,5,2009,3,NaN,1.107001,0.145631,NaN


In [5]:
# create lists for binary, temporal, categorical, continuous variables
temp_vars = ['release_year', 'release_month', 'release_day', 'release_dayofweek']
binary_vars = ['is_collection', 'is_homepage', 'is_English']

cat_vars = [var for var in data.columns
           if data[var].dtypes == 'O' and var not in temp_vars]
num_vars = [var for var in data.columns
           if data[var].dtypes != 'O' and var not in binary_vars and var not in temp_vars
            and var != 'revenue' and var != 'id']

print('The number of binary variables: {}'.format(len(binary_vars)))
print('The number of temporal variables: {}'.format(len(temp_vars)))
print('The number of categorical variables: {}'.format(len(cat_vars)))
print('The number of numerical variables: {}'.format(len(num_vars)))

The number of binary variables: 3
The number of temporal variables: 4
The number of categorical variables: 6
The number of numerical variables: 8


In [6]:
# create a list of the variables that have missing values
vars_with_na = [var for var in data.columns if data[var].isnull().mean() > 0]

print('The number of missing variables: {}'.format(len(vars_with_na)))
print('We have misisng values in {}'.format(vars_with_na))

The number of missing variables: 8
We have misisng values in ['budget', 'runtime', 'num_genres', 'num_pro_comps', 'num_spoken_lang', 'budget_year_ratio', 'runtime_to_mean_year', 'budget_to_mean_year']


In [7]:
# Determine which variables are numerical or which are categorical
cat_na = [var for var in cat_vars if var in vars_with_na]
num_na = [var for var in num_vars if var in vars_with_na]

print('The number of categorical variables with na: ', len(cat_na))
print('The number of numerical variables with na: ', len(num_na))

The number of categorical variables with na:  3
The number of numerical variables with na:  5


In [8]:
# Determine which variables are numerical or which are categorical
cat_not_na = [var for var in cat_vars if var not in cat_na]
num_not_na = [var for var in num_vars if var not in num_na]

print('The number of categorical variables without na: ', len(cat_not_na))
print('The number of numerical variables without na: ', len(num_not_na))

The number of categorical variables without na:  3
The number of numerical variables without na:  3


In [9]:
cat_na

['num_genres', 'num_pro_comps', 'num_spoken_lang']

In [10]:
data.isnull().mean().sort_values(ascending=False)[:8]

budget_to_mean_year     0.270667
budget_year_ratio       0.270667
budget                  0.270667
num_pro_comps           0.052000
num_spoken_lang         0.006667
runtime_to_mean_year    0.004667
runtime                 0.004667
num_genres              0.002333
dtype: float64

## Feature Engineering

We have 3 categorical variables and 5 numerical variables which have missing values. We use different imputation methods for categorical and numerical variables. 

Here, I will explore which method is best for this dataset. In order to do this, we need a baseline model. 

### Split training and test set

We need to split training and test set before using imputation methods. When we engineer features, some techniques learn parameters from data. It is important to learn these parameters only from the train set. This is to avoid over-fitting. 

In [11]:
# log transformation for target
data['revenue'] = np.log1p(data['revenue'])

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(['id','revenue'], axis=1), # drop id and target variable
    data['revenue'], # target
    test_size=0.2, # percentage of test set
    random_state=0 # set seed for reproducibility
)

X_train.shape, X_test.shape

((2400, 21), (600, 21))

### Missing values

#### Categorical variables

In [13]:
data[cat_na].isnull().mean().sort_values(ascending=False)

num_pro_comps      0.052000
num_spoken_lang    0.006667
num_genres         0.002333
dtype: float64

We can see that categorical variables with missing values have small percentage of missing values in data. So, I will replace missing data with the most frequent category in those variables that contain fewere observations without values. 

In [14]:
# replace missing values with frequent category

for var in cat_na:
    
    # there can be more than 1 mode in a variable
    # we take the first one with [0]
    mode = X_train[var].mode()[0]
    
    print(var, mode)
    
    X_train[var].fillna(mode, inplace=True)
    X_test[var].fillna(mode, inplace=True)

num_genres 2
num_pro_comps 1
num_spoken_lang 1


In [15]:
# check that we have no misisng values in the engineered variables
X_train[cat_na].isnull().sum()

num_genres         0
num_pro_comps      0
num_spoken_lang    0
dtype: int64

#### Numerical variables

To engineer missing values in numerical variables, I will:

- add a binary misisng indicator variable
- and then replace the missing values in the original variable with the median

In [16]:
X_train[num_na].head()

,budget,runtime,budget_year_ratio,runtime_to_mean_year,budget_to_mean_year
2370,150000000.0,99.0,37.054018,0.919505,3.809942
1774,35000000.0,122.0,8.741257,1.160886,0.927538
731,4000000.0,108.0,1.033835,0.911392,0.730371
271,72500000.0,116.0,18.016737,1.068036,2.024387
1077,20000000.0,96.0,5.101520,0.858603,1.314708


In [17]:
X_train[num_na].isnull().mean().sort_values(ascending=False)

budget                  0.269167
budget_year_ratio       0.269167
budget_to_mean_year     0.269167
runtime                 0.003750
runtime_to_mean_year    0.003750
dtype: float64

In [18]:
# replace missing values as we described above

for var in num_na:
    
    # calculate the median using the train set
    median_val = X_train[var].median()
    
    print(var, median_val)
    
    # add binary missing indicator (in train and test)
    X_train[var + '_na'] = np.where(X_train[var].isnull(), 1, 0)
    X_test[var + '_na'] = np.where(X_test[var].isnull(), 1, 0)
    
    # replace missing values by the median in train and test
    X_train[var].fillna(median_val, inplace=True)
    X_test[var].fillna(median_val, inplace=True)


budget 16000000.0
runtime 104.0
budget_year_ratio 4.0648041563804895
runtime_to_mean_year 0.9636711281070746
budget_to_mean_year 0.6558648997010403


In [19]:
# check that we have no more missing values in the engineered variables
X_train[vars_with_na].isnull().mean()

budget                  0.0
runtime                 0.0
num_genres              0.0
num_pro_comps           0.0
num_spoken_lang         0.0
budget_year_ratio       0.0
runtime_to_mean_year    0.0
budget_to_mean_year     0.0
dtype: float64

In [20]:
# check that test set does not contain null values 
[var for var in X_test.columns if X_test[var].isnull().mean()>0]

[]

In [21]:
# check the binary missing indicator variables
X_train[['budget_na', 'runtime_na', 'budget_year_ratio_na', 'runtime_to_mean_year_na', 'budget_to_mean_year_na']].head(2)

,budget_na,runtime_na,budget_year_ratio_na,runtime_to_mean_year_na,budget_to_mean_year_na
2370,0,0,0,0,0
1774,0,0,0,0,0


### Numeric variable transformation 

In the previous notebook, we observed that some numerical variables are not normally distributed (`runtime`, `runtime_to_mean_year` are normally distributed). 

So, we will transform with the logarithm the positive numerical variables in order to get a more Gaussian-like distribution

In [22]:
log_trans = ['budget','popularity', 'num_cast', 'budget_year_ratio',
             'popularity_to_mean_year', 'budget_to_mean_year']

In [23]:
for var in log_trans:
    X_train[var] = np.log1p(X_train[var])
    X_test[var] = np.log1p(X_test[var])

In [24]:
# check that train set does not contain null values
[var for var in log_trans if X_train[var].isnull().mean()>0]

[]

In [25]:
# check that test set does not contain null values
[var for var in log_trans if X_test[var].isnull().mean()>0]

[]

### Encoding of categorical variables

We need to transform the strings of the categorical variables into numbers. By doing so, we capture the monotonic relationship between the label and the target

In [26]:
for var in cat_vars:
    print(var)
    print(X_train[var].unique())
    print()

num_genres
['1' '3' '2' '4' '>4']

num_pro_comps
['5' '2' '1' '3' '>5' '4']

num_pro_count
['2' '1' '>2']

num_spoken_lang
['1' '3' '2' '>3']

num_keywords
['<15' '<5' '<10' '<20' '>30']

num_crew
['medium' 'small' 'large' 'missing']



In [27]:
def replace_categories(train, test, y_train, var, target):
    tmp = pd.concat([X_train, y_train], axis=1)
    
    # order the categories in a variable from that with the lowest
    # revenue, to that with the highest
    ordered_labels = tmp.groupby([var])[target].mean().sort_values().index
    
    # create a dictionary of ordered categories to integer values
    ordinal_label = {k: i for i, k in enumerate(ordered_labels, 0)}
    
    print(var, ordinal_label)
    print()
    
    # use the dictionary to replace the categorical strings by integers
    train[var] = train[var].map(ordinal_label)
    test[var] = test[var].map(ordinal_label)

In [28]:
for var in cat_vars:
    replace_categories(X_train, X_test, y_train, var, 'revenue')

num_genres {'1': 0, '2': 1, '3': 2, '4': 3, '>4': 4}

num_pro_comps {'1': 0, '2': 1, '3': 2, '4': 3, '>5': 4, '5': 5}

num_pro_count {'>2': 0, '1': 1, '2': 2}

num_spoken_lang {'1': 0, '2': 1, '3': 2, '>3': 3}

num_keywords {'<5': 0, '<10': 1, '<15': 2, '<20': 3, '>30': 4}

num_crew {'small': 0, 'medium': 1, 'missing': 2, 'large': 3}



In [29]:
# check missing values in train set
[var for var in X_train.columns if X_train[var].isnull().mean()>0]

[]

In [30]:
# check missing values in test set
[var for var in X_test.columns if X_test[var].isnull().mean()>0]

[]

## Feature Scaling

I will use tree-based models later, so feature scaling might not be important here, because we won't use linear models. 

However, I will use MinMaxScaler here.

In [31]:
# create scaler
scaler = MinMaxScaler()

# fit the scaler to the train set
scaler.fit(X_train)

X_train = pd.DataFrame(
    scaler.transform(X_train),
    columns=X_train.columns)

X_test = pd.DataFrame(
    scaler.transform(X_test),
    columns=X_test.columns)

In [32]:
X_train.describe()

,budget,popularity,runtime,is_collection,num_genres,is_homepage,is_English,num_pro_comps,num_pro_count,num_spoken_lang,num_keywords,num_cast,num_crew,release_month,release_day,release_year,release_dayofweek,budget_year_ratio,runtime_to_mean_year,popularity_to_mean_year,budget_to_mean_year,budget_na,runtime_na,budget_year_ratio_na,runtime_to_mean_year_na,budget_to_mean_year_na
count,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000
mean,0.822163,0.347542,0.297704,0.199583,0.374063,0.314167,0.855833,0.314500,0.548958,0.141250,0.222188,0.562832,0.165972,0.527235,0.469847,0.814171,0.538611,0.364054,0.277014,0.230709,0.229799,0.269167,0.003750,0.269167,0.003750,0.269167
std,0.092300,0.129445,0.064728,0.399770,0.274297,0.464280,0.351332,0.301265,0.251297,0.256998,0.270920,0.135655,0.273579,0.308467,0.286422,0.166279,0.224820,0.186777,0.064721,0.121066,0.155886,0.443619,0.061135,0.443619,0.061135,0.443619
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.803649,0.285344,0.253823,0.000000,0.250000,0.000000,1.000000,0.000000,0.500000,0.000000,0.000000,0.491453,0.000000,0.272727,0.233333,0.741935,0.333333,0.258012,0.233684,0.146740,0.133812,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.833832,0.374250,0.284404,0.000000,0.250000,0.000000,1.000000,0.200000,0.500000,0.000000,0.250000,0.560339,0.000000,0.545455,0.466667,0.860215,0.666667,0.356279,0.264315,0.236521,0.205218,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.866808,0.435091,0.327217,0.000000,0.500000,1.000000,1.000000,0.400000,0.500000,0.333333,0.250000,0.636614,0.333333,0.818182,0.700000,0.935484,0.666667,0.467522,0.307237,0.312301,0.293678,1.000000,0.000000,1.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Conclusion

In [35]:
# Let's now save the train and test sets for the next step
X_train.to_csv(path/'xtrain.csv', index=False)
X_test.to_csv(path/'xtest.csv', index=False)

y_train.to_csv(path/'ytrain.csv', index=False)
y_test.to_csv(path/'ytest.csv', index=False)

In [36]:
# let's ave the scaler
joblib.dump(scaler, 'minmax_scaler.joblib')

['minmax_scaler.joblib']